In [37]:
import numpy as np
import pandas as pd
from pathlib import Path
import gc
import sys
import json
import warnings

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import RobustScaler

sys.path.append(str(Path().resolve().parent))

from src.utils import (
    seed_every_thing,
    fetch_data,
    Config,
    plot_metric,
    reduce_tf_gpu_memory,
    reduce_mem_usage,
    fetch_custom_data,
    CustomL1Loss
)
warnings.simplefilter('ignore')
pd.options.display.max_columns = 200

In [38]:
def find_R(n):
    if n in [0, 1, 2]:
        return 0
    elif n in [3, 4, 5]:
        return 2
    elif n in [6, 7, 8]:
        return 1
    else:
        raise Exception()

def find_C(n):
    if n in [0, 3, 6]:
        return 0
    elif n in [1, 4, 7]:
        return 1
    elif n in [2, 5, 8]:
        return 2
    else:
        raise Exception()
        
def build_model(config: Config, n_features) -> keras.models.Sequential:
    model = keras.models.Sequential([keras.layers.Input(shape=(config.cut, n_features))])
    for n_unit in config.n_units:
        model.add(
            keras.layers.Bidirectional(
                keras.layers.LSTM(
                    n_unit,
                    return_sequences=True,
                )
            )
        )
    for n_unit in config.n_dense_units:
        model.add(keras.layers.Dense(n_unit, activation="selu"))
    model.add(keras.layers.Dense(1))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=config.lr), loss="mae")
    return model


def compute_metric(df, merge=True):
    y_true, y_pred, u_out = df["pressure"].values, df["pred"].values, df["u_out"].values
    w = (1 - u_out)
    
    if merge:
        mae = w * np.abs(y_true - y_pred)
        mae = np.sum(mae) / np.sum(w)
        return mae
    else:
        y_true, y_pred, w = y_true.reshape(-1, 35), y_pred.reshape(-1, 35), w.reshape(-1, 35)
        maes = []
        for yt, yp, w_ in zip(y_true, y_pred, w):
            mae = w_ * np.abs(yt - yp)
            mae = np.sum(mae) / np.sum(w_)
            maes.append(mae)
        return np.array(maes)

In [39]:
basedir = Path().resolve().parent
logdir = basedir / "logs"
datadir = basedir / "data"
cachedir = basedir / "cache"

In [40]:
cfg_file_path = logdir / "lstm-less-addfeatures-cut-large-custom-mae" / "config.json"
with open(cfg_file_path, "rb") as f:
    config = json.load(f)
config = Config().update(config)

In [41]:
# rc_preds = pd.read_csv(logdir / "cnn-classify-rc-reliable" / "valid_preds.csv")
# rc_test_preds = pd.read_csv(logdir / "cnn-classify-rc-reliable" / "test_preds.csv")

rc_preds = pd.read_csv(logdir / "cnn-classify-rc-reliable_3" / "valid_preds.csv")
rc_test_preds = pd.read_csv(logdir / "cnn-classify-rc-reliable_3" / "test_preds.csv")

In [7]:
valid_preds = pd.read_csv(logdir / "lstm-less-addfeatures-cut-large-custom-mae" / "valid_preds.csv", index_col=0)

In [9]:
train = pd.read_csv(cachedir / "train_lstm-less-cut-addfeatures_debugFalse.csv")
train.head()

,time_step,u_in,u_out,pressure,kfold,count,corss,cross2,cross3,time_delta,time_step_cumsum,u_in_cumsum,u_in_cummean,u_in_lag_b1,u_out_lag_b1,u_in_lag_b2,u_out_lag_b2,u_in_lag_b3,u_out_lag_b3,u_in_lag_b4,u_out_lag_b4,u_in_lag_b5,u_out_lag_b5,u_in_lag_f1,u_out_lag_f1,u_in_lag_f2,u_out_lag_f2,u_in_lag_f3,u_out_lag_f3,u_in_lag_f4,u_out_lag_f4,u_in_lag_f5,u_out_lag_f5,u_in_diff_b1,u_out_diff_b1,u_in_diff_b2,u_out_diff_b2,u_in_diff_b3,u_out_diff_b3,u_in_diff_b4,u_out_diff_b4,u_in_diff_b5,u_out_diff_b5,u_in_diff_f1,u_out_diff_f1,u_in_diff_f2,u_out_diff_f2,u_in_diff_f3,u_out_diff_f3,u_in_diff_f4,u_out_diff_f4,u_in_diff_f5,u_out_diff_f5,u_in_bwindow_mean,u_in_bwindow_max,u_in_bwindow_min,u_in_bwindow_std,u_in_fwindow_mean,u_in_fwindow_max,u_in_fwindow_min,u_in_fwindow_std,u_in_cwindow_mean,u_in_cwindow_max,u_in_cwindow_min,u_in_cwindow_std,u_in_bwindow_ewm,u_in_fwindow_ewm,u_in_cwindow_ewm,u_in_bwindow_mean_diff,u_in_bwindow_max_diff,u_in_bwindow_min_diff,u_in_fwindow_mean_diff,u_in_fwindow_max_diff,u_in_fwindow_min_diff,u_in_cwindow_mean_diff,u_in_cwindow_max_diff,u_in_cwindow_min_diff,area,area_insp,area_insp_last,u_in_max,u_in_mean,u_in_std,area_max,area_insp_max,area_mean,area_insp_mean,vibs,RC_20_10,RC_20_20,RC_20_50,RC_50_10,RC_50_20,RC_50_50,RC_5_10,RC_5_20,RC_5_50,R_20,R_5,R_50,C_10,C_20,C_50,norm_time_step,u_in_max_diff,area_max_diff,area_insp_max_diff,u_in_mean_diff,area_mean_diff,area_insp_mean_diff
0,0.00000,0.0833,0,5.836,4,1,0.0,0.00000,0.0,0.00000,0.00000,0.0833,0.0833,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,18.39,0.0,22.52,0.0,22.81,0.0,25.36,0.0,27.27,0.0,0.0000,0.0,0.000,0.0,0.000,0.0,0.00,0.0,0.0,0.0,-18.3000,0.0,-22.420,0.0,-22.720,0.0,-25.270,0.0,-27.170,0.0,0.013885,0.0833,0.0,0.03105,19.40,27.27,0.0833,9.0600,5.120,22.52,0.0,8.91,0.05957,26.45,20.69,0.06946,0.0,0.0833,-19.310,-27.170,0.0,-5.040,-22.420,0.0833,0.0000,0.0000,22.6,28.31,18.95,9.52,22.6,22.6,13.58,13.58,13,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.00000,-28.230,-22.60,-22.60,-18.8800,-13.580,-13.580
1,0.03366,18.3900,0,5.906,4,2,0.0,0.03366,0.0,0.03366,0.03366,18.4700,9.2340,0.0833,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,22.52,0.0,22.81,0.0,25.36,0.0,27.27,0.0,27.12,0.0,18.3000,0.0,0.000,0.0,0.000,0.0,0.00,0.0,0.0,0.0,-4.1250,0.0,-4.426,0.0,-6.973,0.0,-8.875,0.0,-8.740,0.0,3.078000,18.3900,0.0,6.84400,23.90,27.27,18.3900,3.0920,7.973,22.81,0.0,10.34,13.16000,26.94,22.34,15.30500,0.0,18.3900,-5.523,-8.875,0.0,10.410,-4.426,18.3900,0.6187,0.6187,22.6,28.31,18.95,9.52,22.6,22.6,13.58,13.58,13,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.02907,-9.930,-21.97,-21.97,-0.5767,-12.960,-12.960
2,0.06750,22.5200,0,7.875,4,3,0.0,0.06750,0.0,0.03387,0.10120,40.9700,13.6600,18.3900,0.0,0.0833,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,22.81,0.0,25.36,0.0,27.27,0.0,27.12,0.0,26.81,0.0,4.1250,0.0,22.420,0.0,0.000,0.0,0.00,0.0,0.0,0.0,-0.2996,0.0,-2.846,0.0,-4.750,0.0,-4.617,0.0,-4.297,0.0,6.830000,22.5200,0.0,9.70000,25.31,27.27,22.5200,1.9770,11.140,25.36,0.0,11.26,19.84000,26.84,24.69,15.68000,0.0,22.5200,-2.803,-4.750,0.0,11.370,-2.846,22.5200,1.3810,1.3810,22.6,28.31,18.95,9.52,22.6,22.6,13.58,13.58,13,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.05832,-5.805,-21.22,-21.22,3.5490,-12.195,-12.195
3,0.10156,22.8100,0,11.740,4,4,0.0,0.10156,0.0,0.03403,0.20280,63.7800,15.9450,22.5200,0.0,18.3900,0.0,0.0833,0.0,0.0000,0.0,0.0,0.0,25.36,0.0,27.27,0.0,27.12,0.0,26.81,0.0,27.86,0.0,0.2996,0.0,4.426,0.0,22.720,0.0,0.00,0.0,0.0,0.0,-2.5470,0.0,-4.450,0.0,-4.320,0.0,-3.998,0.0,-5.055,0.0,10.630000,22.8100,0.0,10.69500,26.20,27.86,22.8100,1.7000,14.550,27.27,0.0,11.49,21.97000,27.58,26.69,12.18000,0.0,22.8100,-3.395,-5.055,0.0,8.260,-4.450,22.8100,2.1560,2.1560,22.6,28.31,18.95,9.52,22.6,22.6,13.58,13.58,13,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0.08770,-5.504,-20.44,-20.44,3.8500,-11.420,-11.420
4,0.13570,25.3600,0,12.234,4,5,0.0,0.13570,0.0,0.03420,0.33840,89.1000,17.8300,22.8100,0.0,22.5200,0.0,18.3900,0.0,0.0833,0.0,0.0,0.0,27.27,0.0,27.12,0.0,26.81,0.0,27.86,0.0,28.31,0.0,2.5470,0.0,2.846,0.0,6.973,0.0,2

In [16]:
rc_cols = [c for c in train.columns if "RC_" in c]
df = train[["u_out", "pressure"] + rc_cols]
df["RC"] = df[rc_cols].values.argmax(axis=1)
df.drop(rc_cols, axis=1, inplace=True)
df["pred"] = valid_preds.values.reshape(-1)
df.head()

,u_out,pressure,RC,pred
0,0,5.836,2,5.797
1,0,5.906,2,5.880
2,0,7.875,2,7.906
3,0,11.740,2,11.870
4,0,12.234,2,12.370


In [42]:
df["RC_pred"] = np.tile(rc_preds, (1, 35)).reshape(-1, rc_preds.shape[1]).argmax(axis=1)

In [43]:
print(compute_metric(df))

print(compute_metric(df[df.RC == df.RC_pred]))

print(compute_metric(df[df.RC == df.RC_pred]))

0.16571056865473463
0.15291127172745936
0.15291127172745936


In [53]:
test = pd.read_csv(cachedir / "test_lstm-less-cut-addfeatures_debugFalse.csv")
test.head()

,time_step,u_in,u_out,count,pressure,corss,cross2,cross3,time_delta,time_step_cumsum,u_in_cumsum,u_in_cummean,u_in_lag_b1,u_out_lag_b1,u_in_lag_b2,u_out_lag_b2,u_in_lag_b3,u_out_lag_b3,u_in_lag_b4,u_out_lag_b4,u_in_lag_b5,u_out_lag_b5,u_in_lag_f1,u_out_lag_f1,u_in_lag_f2,u_out_lag_f2,u_in_lag_f3,u_out_lag_f3,u_in_lag_f4,u_out_lag_f4,u_in_lag_f5,u_out_lag_f5,u_in_diff_b1,u_out_diff_b1,u_in_diff_b2,u_out_diff_b2,u_in_diff_b3,u_out_diff_b3,u_in_diff_b4,u_out_diff_b4,u_in_diff_b5,u_out_diff_b5,u_in_diff_f1,u_out_diff_f1,u_in_diff_f2,u_out_diff_f2,u_in_diff_f3,u_out_diff_f3,u_in_diff_f4,u_out_diff_f4,u_in_diff_f5,u_out_diff_f5,u_in_bwindow_mean,u_in_bwindow_max,u_in_bwindow_min,u_in_bwindow_std,u_in_fwindow_mean,u_in_fwindow_max,u_in_fwindow_min,u_in_fwindow_std,u_in_cwindow_mean,u_in_cwindow_max,u_in_cwindow_min,u_in_cwindow_std,u_in_bwindow_ewm,u_in_fwindow_ewm,u_in_cwindow_ewm,u_in_bwindow_mean_diff,u_in_bwindow_max_diff,u_in_bwindow_min_diff,u_in_fwindow_mean_diff,u_in_fwindow_max_diff,u_in_fwindow_min_diff,u_in_cwindow_mean_diff,u_in_cwindow_max_diff,u_in_cwindow_min_diff,area,area_insp,area_insp_last,u_in_max,u_in_mean,u_in_std,area_max,area_insp_max,area_mean,area_insp_mean,vibs,RC_20_10,RC_20_20,RC_20_50,RC_50_10,RC_50_20,RC_50_50,RC_5_10,RC_5_20,RC_5_50,R_20,R_5,R_50,C_10,C_20,C_50,norm_time_step,u_in_max_diff,area_max_diff,area_insp_max_diff,u_in_mean_diff,area_mean_diff,area_insp_mean_diff
0,0.00000,0.000,0,1,0,0.0,0.00000,0.0,0.00000,0.0000,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,7.516,0.0,14.65,0.0,21.23,0.0,26.33,0.0,30.48,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,-7.516,0.0,-14.650,0.0,-21.230,0.0,-26.330,0.0,-30.48,0.0,0.000,0.000,0.0,0.000,16.70,30.48,0.000,10.570,2.771,14.65,0.0,5.120,0.00,28.69,12.695,0.00,0.0,0.000,-16.700,-30.48,0.0,-2.771,-14.650,0.000,0.0000,0.0000,19.48,37.53,17.45,14.77,19.48,19.48,13.09,13.09,3,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0.00000,-37.53,-19.48,-19.48,-17.450,-13.09,-13.09
1,0.03190,7.516,0,2,0,0.0,0.03190,0.0,0.03190,0.0319,7.516,3.758,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,14.650,0.0,21.23,0.0,26.33,0.0,30.48,0.0,33.53,0.0,7.516,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,-7.137,0.0,-13.720,0.0,-18.810,0.0,-22.970,0.0,-26.03,0.0,1.253,7.516,0.0,2.800,22.30,33.53,7.516,9.016,5.426,21.23,0.0,7.797,5.37,32.16,19.330,6.26,0.0,7.516,-14.770,-26.03,0.0,2.090,-13.720,7.516,0.2397,0.2397,19.48,37.53,17.45,14.77,19.48,19.48,13.09,13.09,3,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0.02942,-30.03,-19.25,-19.25,-9.940,-12.85,-12.85
2,0.06384,14.650,0,3,0,0.0,0.06384,0.0,0.03192,0.0957,22.170,7.390,7.516,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,21.230,0.0,26.33,0.0,30.48,0.0,33.53,0.0,35.72,0.0,7.137,0.0,14.65,0.0,0.00,0.0,0.00,0.0,0.0,0.0,-6.580,0.0,-11.670,0.0,-15.836,0.0,-18.890,0.0,-21.06,0.0,3.695,14.650,0.0,5.617,27.00,35.72,14.650,7.270,8.710,26.33,0.0,10.050,12.01,34.72,24.770,10.96,0.0,14.650,-12.340,-21.06,0.0,5.938,-11.670,14.650,0.7075,0.7075,19.48,37.53,17.45,14.77,19.48,19.48,13.09,13.09,3,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0.05884,-22.89,-18.78,-18.78,-2.799,-12.38,-12.38
3,0.09576,21.230,0,4,0,0.0,0.09576,0.0,0.03192,0.1915,43.400,10.850,14.650,0.0,7.516,0.0,0.000,0.0,0.0,0.0,0.0,0.0,26.330,0.0,30.48,0.0,33.53,0.0,35.72,0.0,36.97,0.0,6.580,0.0,13.72,0.0,21.23,0.0,0.00,0.0,0.0,0.0,-5.090,0.0,-9.260,0.0,-12.310,0.0,-14.484,0.0,-15.74,0.0,7.234,21.230,0.0,8.250,30.72,36.97,21.230,5.500,12.520,30.48,0.0,11.664,18.61,36.34,29.220,14.00,0.0,21.230,-9.484,-15.74,0.0,8.700,-9.260,21.230,1.3850,1.3850,19.48,37.53,17.45,14.77,19.48,19.48,13.09,13.09,3,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0.08826,-16.31,-18.11,-18.11,3.780,-11.71,-11.71
4,0.12770,26.330,0,5,0,0.0,0.12770,0.0,0.03190,0.3190,69.700,13.945,21.230,0.0,14.650,0.0,7.516,0.0,0.0,0.0,0.0,0.0,30.480,0.0,33.53,0.0,35.72,0.0,36.97,0.0,37.53,0.0,5.090,0.0,11.67,0.0,18.81,0.0,26.33,0.0,0.0,0.0,-4.164,0.0,-7.227,0.0,-9.400,0.0,-10.650,0.0,-11.22,0.0,11.620,26.330,0.0,10.040,33.44,37.53,26.330,3.955,16.720,33.53,0.0,12.414,24.12,37.22,32.600

In [54]:
train_ = train.copy()

In [56]:
features = np.array(list(train.drop(["kfold", "pressure"], axis=1).columns))
cont_features = [
    f
    for f in features
    if ("RC_" not in f) and ("u_out" not in f) and ("R_" not in f) and ("C_" not in f) and ("u_out" not in f)
]

RS = RobustScaler()
train[cont_features] = RS.fit_transform(train[cont_features])
test[cont_features] = RS.transform(test[cont_features])

In [57]:
train_data, test_data = train[features].values, test[features].values
train_data = train_data.reshape(-1, config.cut, train_data.shape[-1])
targets = train[["pressure"]].to_numpy().reshape(-1, config.cut)
test_data = test_data.reshape(-1, config.cut, test_data.shape[-1])

In [58]:
kfolds = train.iloc[0 :: config.cut]["kfold"].values
# reliable = train.iloc[0 :: config.cut]["is_reliable"].values

In [59]:
rc_feature_idxs = np.array([i for i, f in enumerate(features) if "RC_" in f])
r_feature_idxs = np.array([i for i, f in enumerate(features) if f in ["R_20", "R_5", "R_50"]])
c_feature_idxs = np.array([i for i, f in enumerate(features) if f in ["C_10", "C_20", "C_50"]])

In [61]:
rc_preds_onehot = np.eye(rc_preds.shape[1])[rc_preds.values.argmax(axis=1)]
r_preds_onehot = np.eye(3)[np.array(list(map(find_R, rc_preds.values.argmax(axis=1))))]
c_preds_onehot = np.eye(3)[np.array(list(map(find_C, rc_preds.values.argmax(axis=1))))]

In [62]:
rc_true_onehot = train.iloc[0::35][features[rc_feature_idxs]].values

In [63]:
train_data_ = train_data.copy()

In [107]:
train_data_[:, :, rc_feature_idxs] = np.tile(rc_preds_onehot, (1, 35)).reshape(-1, config.cut, rc_preds_onehot.shape[1])
train_data_[:, :, r_feature_idxs] = np.tile(r_preds_onehot, (1, 35)).reshape(-1, config.cut, r_preds_onehot.shape[1])
train_data_[:, :, c_feature_idxs] = np.tile(c_preds_onehot, (1, 35)).reshape(-1, config.cut, c_preds_onehot.shape[1])

In [110]:
def inference(train_data):
    with tf.device(f"/GPU:2"):
        valid_preds = np.empty_like(targets)

        for fold in range(config.n_splits):
            train_idx, test_idx = (kfolds != fold), (kfolds == fold)
            # test_idx = np.logical_and(test_idx, ~reliable)
            print("-" * 15, ">", f"Fold {fold+1}", "<", "-" * 15)
            savedir = logdir / "lstm-less-addfeatures-cut-large-custom-mae" / f"fold{fold}"

            X_valid, y_valid = train_data[test_idx], targets[test_idx]

            model = build_model(config=config, n_features=len(features))
            model.load_weights(savedir / "weights_custom_best.h5")

            valid_preds[test_idx, :] = model.predict(X_valid).squeeze()

            del model, X_valid, y_valid
            keras.backend.clear_session()
            gc.collect()
    return valid_preds

In [69]:
plane_preds = inference(train_data)

--------------- > Fold 1 < ---------------
--------------- > Fold 2 < ---------------
--------------- > Fold 3 < ---------------
--------------- > Fold 4 < ---------------
--------------- > Fold 5 < ---------------
--------------- > Fold 6 < ---------------
--------------- > Fold 7 < ---------------
--------------- > Fold 8 < ---------------
--------------- > Fold 9 < ---------------
--------------- > Fold 10 < ---------------


In [112]:
df = train[["pressure", "u_out"]].copy()
df["pred"] = plane_preds.reshape(-1)
compute_metric(df)

0.1658453562466597

In [73]:
mod_preds = inference(train_data_)

--------------- > Fold 1 < ---------------
--------------- > Fold 2 < ---------------
--------------- > Fold 3 < ---------------
--------------- > Fold 4 < ---------------
--------------- > Fold 5 < ---------------
--------------- > Fold 6 < ---------------
--------------- > Fold 7 < ---------------
--------------- > Fold 8 < ---------------
--------------- > Fold 9 < ---------------
--------------- > Fold 10 < ---------------


In [111]:
df = train[["pressure", "u_out"]].copy()
df["pred"] = mod_preds.reshape(-1)
compute_metric(df)

0.23818726887940972

In [153]:
thr = 0.99999
insert_idx = (np.max(rc_preds.values, axis=1) > thr)
insert_idx = np.tile(insert_idx, (35, 1)).T.reshape(-1)
print(np.round(insert_idx.sum() / insert_idx.shape[0], 3))

0.244


In [154]:
df = train[["pressure", "u_out"]].copy()
df["pred"] = np.where(insert_idx, mod_preds.reshape(-1), plane_preds.reshape(-1))
compute_metric(df)

0.16603730190284685